In [ ]:
import sys, os

from RecHit import RecHit
from RecTrack import RecTrack
from TimestampsTPMatching import * 
from Rates import *
from Plots import *

In [ ]:
sys.path.append('/afs/cern.ch/work/l/lmozzina/public')
run = 'example_data_cosmic_rays' 
d = __import__(run)

time_window_TP = 15 # ns

plots_dir = f"Plots_{run}_{time_window_TP}ns"

if not os.path.exists(plots_dir):
    os.makedirs(plots_dir)

In [ ]:
# hits = GetGlobalTimestampsHit(d.hits,'light',0) 
hits = GetGlobalTimestampsHit(d.hits,'light',1e3)

tpsX = GetGlobalTimestampsTP(d.tpgs_mb1)
tpsY = GetGlobalTimestampsTP(d.tpgs_mb2)

matched_tp_pairs_X = MatchTPsSameBX(tpsX, tpsY, 3, 4, time_window_TP) # L with H match, +/- 15 ns time window
matched_tp_pairs_Y = MatchTPsSameBX(tpsY, tpsX, 3, 4, time_window_TP) # L with H match, +/- 15 ns time window

In [ ]:
PlotLayerRates(hits, run, plots_dir)
PlotCellRates(hits, run, plots_dir)

In [ ]:
RecTracks_X = [] 
RecTracks_Y = []

n_hits_X = []
n_hits_Y = []

slope_X = []
slope_Y = []

XIntercept_X = []
XIntercept_Y = []

ChiSquare_X = []
ChiSquare_Y = []

Residual_X = []
Residual_Y = []

old_index_X = 0
old_index_Y = 0
for pair in matched_tp_pairs_X:
    rec_hits_X = []
    
    tpx = pair[0]
    tpy = pair[1]
    tpy_ts = tpsY[tpy]['timestamp'] # time match with TPG y
    old_index_X, matched_hits_X = SelectHits(tpy_ts, hits, 0, -100, 500, 250, old_index_X) 
        
    for index in matched_hits_X:
        st, ly, wi = hits[index]['st'], hits[index]['ly'], hits[index]['wi']
        hit_ts = hits[index]['timestamp']
        tdiff = hit_ts - tpy_ts                    # here we assign the hit timestamp to compute its position. tp time is reference
        rec_hit = RecHit(tdiff*1E9, st, ly, wi)
        rec_hit.SetHitPosition()
        rec_hits_X.append(rec_hit) 
         
    rec_track_X = RecTrack(rec_hits_X, tpsX[tpx])
    rec_track_X.FitTPLateralities()
    n_hits_X.append(rec_track_X.nHits)

    slope_X.append([rec_track_X.Slope, rec_track_X.nHits])
    XIntercept_X.append([rec_track_X.XIntercept, rec_track_X.nHits])
    ChiSquare_X.append([rec_track_X.ChiSquare, rec_track_X.nHits])
    
    Residual_X.append([rec_track_X.Residual_Hit_Wire_Distance(), rec_track_X.nHits])
    
    RecTracks_X.append(rec_track_X)  

for pair in matched_tp_pairs_Y:
    rec_hits_Y = []
    
    tpx = pair[1]
    tpy = pair[0]
    tpx_ts = tpsX[tpx]['timestamp']
    old_index_Y, matched_hits_Y = SelectHits(tpx_ts, hits, 1, -100, 500, 250, old_index_Y)  
        
    for index in matched_hits_Y:
        st, ly, wi = hits[index]['st'], hits[index]['ly'], hits[index]['wi']
        hit_ts = hits[index]['timestamp']
        tdiff = hit_ts-tpx_ts
        rec_hit = RecHit(tdiff*1E9, st, ly, wi)
        rec_hit.SetHitPosition()
        rec_hits_Y.append(rec_hit)

    rec_track_Y = RecTrack(rec_hits_Y, tpsY[tpy]) 
    rec_track_Y.FitTPLateralities()
    n_hits_Y.append(rec_track_Y.nHits)
    
    slope_Y.append([rec_track_Y.Slope, rec_track_Y.nHits])
    XIntercept_Y.append([rec_track_Y.XIntercept, rec_track_Y.nHits])
    ChiSquare_Y.append([rec_track_Y.ChiSquare, rec_track_Y.nHits])
    
    Residual_Y.append([rec_track_Y.Residual_Hit_Wire_Distance(), rec_track_Y.nHits])
    
    RecTracks_Y.append(rec_track_Y)  

In [ ]:
PlotHits(n_hits_X, n_hits_Y, run, plots_dir)

In [ ]:
PlotSlopes(slope_X, slope_Y, run, plots_dir)
PlotSlopes(slope_X, slope_Y, run, plots_dir, 3)
PlotSlopes(slope_X, slope_Y, run, plots_dir, 4)

In [ ]:
PlotXIntercepts(XIntercept_X, XIntercept_Y, run, plots_dir)
PlotXIntercepts(XIntercept_X, XIntercept_Y, run, plots_dir, 3)
PlotXIntercepts(XIntercept_X, XIntercept_Y, run, plots_dir, 4)

In [ ]:
PlotChiSquares(ChiSquare_X, ChiSquare_Y, run, plots_dir)
PlotChiSquares(ChiSquare_X, ChiSquare_Y, run, plots_dir, 3)
PlotChiSquares(ChiSquare_X, ChiSquare_Y, run, plots_dir, 4)

In [ ]:
PlotResiduals(Residual_X, Residual_Y, run, plots_dir)
PlotResiduals(Residual_X, Residual_Y, run, plots_dir, 3)
PlotResiduals(Residual_X, Residual_Y, run, plots_dir, 4)

In [ ]:
PlotResidualsVsDistance(Residual_X, Residual_Y, run, plots_dir)

In [ ]:
c_values = 500
m_values = 1000

for track in RecTracks_X:
    track.HoughFit(c_values, m_values)
    track.PlotFit()
    
for track in RecTracks_Y:
    track.HoughFit(c_values, m_values)
    track.PlotFit()